In [2]:
!pip install findspark

In [3]:
import findspark # 환경변수를 읽어줌 # jvm경로  다 알고잇음

In [4]:
findspark.init()

In [5]:
from pyspark import SparkContext

In [6]:
sc = SparkContext()

In [7]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [8]:
sc = SparkContext.getOrCreate() #있으면 가져오고 없으면 만들고

In [20]:
sc.stop()

In [11]:
sc = SparkContext(master="local[2]",appName="Spark Test")

In [12]:
sc

<SparkContext master=local[2] appName=Spark Test>

In [13]:
from pyspark import SparkConf

In [90]:
conf = SparkConf()

In [91]:
conf = conf.setMaster("local[2]").setAppName('Conf')

In [92]:
sc = SparkContext(conf=conf)

In [93]:
sc

<SparkContext master=local[2] appName=Conf>

In [23]:
#실제 환경에서 할때는 위에서처럼 한다

In [26]:
rdd = sc.parallelize([1,2,3,4,5])

In [28]:
rdd.collect() 

[1, 2, 3, 4, 5]

In [29]:
rdd.glom().collect() #glom : 파티션 마다 작업을한다. 1,2,3,4,5 가 조각이나서 작업이 수행됨. 

[[1, 2], [3, 4, 5]]

In [30]:
text = sc.parallelize("./sample/text-test.txt")

In [31]:
text.glom().collect() #파티션을 나눠서 RDD 객체를 생성함.

[['.', '/', 's', 'a', 'm', 'p', 'l', 'e', '/', 't', 'e'],
 ['x', 't', '-', 't', 'e', 's', 't', '.', 't', 'x', 't']]

In [32]:
text.collect()

['.',
 '/',
 's',
 'a',
 'm',
 'p',
 'l',
 'e',
 '/',
 't',
 'e',
 'x',
 't',
 '-',
 't',
 'e',
 's',
 't',
 '.',
 't',
 'x',
 't']

# glom 은 transformation collect 는 action 

In [94]:
rdd = sc.parallelize(['b','a','c'],3)

In [95]:
rddMap = rdd.map(lambda x : (x,1))

In [96]:
rddMap.collect()

[('b', 1), ('a', 1), ('c', 1)]

map한결과를 가지고온다.
키값으로 sorting 하면 a,b,c, 순으로 나온다

In [39]:
sorted(rddMap.glom().collect())

[[('a', 1)], [('b', 1)], [('c', 1)]]

In [41]:
#rdd 에있는것을 filter 함.

rdd= sc.parallelize([1,2,3,4,5])
rdd.filter(lambda x:x%2==0).collect()

[2, 4]

In [48]:
#flatMap 은 map 과 유사하지만, 리스트가 아닌 평면된 결과를 리턴. 
rdd = sc.parallelize([2,3,4])

In [49]:
rdd1 = rdd.map(lambda x:range(1,x) )

In [50]:
rdd.glom().collect()

[[2], [3, 4]]

In [51]:
rdd.flatMap(lambda x:range(1,x)).collect() #map 말고 flatmap 할 경우.

[1, 1, 2, 1, 2, 3]

In [63]:
rdd = sc.parallelize(["Roses are Red","Violets are blue"])

In [57]:
rdd.map(lambda x:x.split()).collect()

[['Roses', 'are', 'Red'], ['Violets', 'are', 'blue']]

In [59]:
rdd.flatMap(lambda x:x.split()).collect()

['Roses', 'are', 'Red', 'Violets', 'are', 'blue']

In [61]:
sc.parallelize([1,1,2,3]).distinct().collect() #1123을 중복한거 없엔결과를 리턴한다 . 

[2, 1, 3]

In [65]:
rdd.flatMap(lambda x:x.split()).distinct().collect()

['Roses', 'are', 'Red', 'Violets', 'blue']

#sample 하면 일부데이터만 rdd set 으로 가져온다 . 중복허용여부, 크기 비율 , 전체데이터를 1이라고 봤을때 몇퍼센트로 볼것인지. 

In [66]:
rdd = sc.parallelize(range(100),4)

In [67]:
subset = rdd.sample(False,0.1,12)

In [68]:
subset.collect()

[8, 10, 15, 16, 19, 26, 32, 62, 63, 65, 72, 98]

In [69]:
len(subset.collect())

12

In [71]:
subset.glom().collect()

[[8, 10, 15, 16, 19], [26, 32], [62, 63, 65, 72], [98]]

In [72]:
# 두 데이터를 석어서 하나의 셋으로


In [100]:
x = sc.parallelize([("a",1),("b",4)])
y = sc.parallelize([("a",2),("b",4)])

In [101]:
#두개의 RDD 만듬


In [102]:
sorted(x.leftOuterJoin(y).collect())

[('a', (1, 2)), ('b', (4, 4))]

In [ ]:
# rightOuterJoin 과 lefOuterJoin 은 기준만 다를 뿐 같다.

In [ ]:
#join : 키값이 같은것들에 대해 join
#intersection : 교집합해줌.


In [83]:
sorted(x.intersection(y).collect())

[('b', 4)]

In [85]:
rdd.count()

100

In [86]:
rdd.glom().count() # 4개의 파티션

4

In [87]:
rdd.repartition(10).glom().count() # 10개 파티션으로 

10

In [88]:
#나중에 파티션 갯수 일치시켜줄경우에 사용한다 .

In [89]:
sc.stop()

In [103]:
rdd.takeSample(False,10,123) # 앞에서나온 sample 함수는 얘가 action 이 아니라서 collect 랑 같이썼다.


['c', 'a', 'b']

reduce 를 하면 key value 쌍에서 value 를 어떻게 처리할 것인지

In [104]:
#reduce 를 하려면 최소한 키벨류쌍으로 된 무언가가 있어야한다. 


In [105]:
rdd = sc.parallelize([1,2,3,4,5],2)

In [106]:
rdd.reduce(lambda x,y: x+y)

15

In [107]:
#glom 을 하면 다르다 , 파티션 단위로 하기때문에
rdd.glom().collect()

#1,2 더하기 , 3,4 더한것을 5와 더하고 전체를 더함. 



[[1, 2], [3, 4, 5]]

In [108]:
rdd.reduce(lambda x,y:x+y)

15

In [109]:
rdd = sc.parallelize([1,2,.5,.1,5,.2],2)

In [110]:
rdd.reduce(lambda x,y:x/y)

10.0

In [111]:
#파티션갯수 바꾸면 예상결과가 안나옴 (나눗셈, 뺄셈 의 경우)
rdd = sc.parallelize([1,2,.5,.1,5,.2],3)
rdd.reduce(lambda x,y:x/y)

0.004

In [112]:
rdd.glom().collect()

[[1, 2], [0.5, 0.1], [5, 0.2]]

In [113]:
#파티션갯수를 잘맞춰주는것이 중요하다 , 특히 뺼셈 나누셈 조심하기.
#어떤 단어가 몇번나왔는지 리듀스를 통해서 뽑아줄수있다.


In [115]:
rdd = sc.parallelize([("a",1),("b",2),("a",3)])

In [ ]:
#두개 가져와서 더하고 처리

In [116]:
from operator import add #튜플일때 key value 쌍을 알아서 더해줌. 

In [117]:
rdd.reduceByKey(add).collect()

[('b', 2), ('a', 4)]

In [ ]:
#파티션으로 나뉜 애들이 어ㅓㄸㅎ게 저장되는지 보는것.


In [118]:
rdd.saveAsTextFile("abc") # 텍스트파일로 저장

지금은 standalone 이지만
작업하는 애들마다 마스터와 슬레이브로 구성될 것

In [119]:
fromText  = sc.parallelize("abc")

In [120]:
fromText.collect()

['a', 'b', 'c']

In [121]:
#element 3개를 불러옴


In [122]:
fromText.glom().collect()

[['a'], ['b', 'c']]

In [123]:
#실제 파일이 분산되어있는거랑 ,파티션이랑은 다른얘기

In [124]:
fromText=  sc.textFile("abc") #텍스트불러오기

In [125]:
fromText.glom().collect() # 두개로 분산된것을 가져옴.

[["('a', 1)"], ["('b', 2)", "('a', 3)"], []]

In [126]:
fromText= fromText.repartition(2)

In [131]:
data = sc.parallelize([("a",22),{"b":23},["c",4]]) # 튜플딕셔너리리스트 섞임

In [132]:
obj= data.collect()

In [134]:
type(obj)

list

In [135]:
obj[1]["b"] #dict니까

23

파이썬이랑 hyperj 랑 jvm 메모리에 생성된 객체와 파이썬객체와 통신한다

이런과정이 많아지면 느려진다

코드는 파이썬에서 작업하고있지만, 파이썬에서 메모리를 잡은게아니다, jvm 에서 메모리를 잡은것이다. 통신해서 가져온것뿐이다.
obj가 파이썬에서 받아줌

In [137]:
from pyspark.sql import SparkSession

In [152]:
spark = SparkSession.builder.master("local[2]").appName("Session").getOrCreate()

In [153]:
spark

그래도 생성이 안됨, 위에있던 sc 랑 같다. 

In [ ]:
#스파크 컨텍스트 드라이버를 로드했었다. ㅇ

In [144]:
sc = SparkContext.getOrCreate()

In [154]:
#어차피 쓰는 드라이버는 하나다 . 위를 쓰든 아래를쓰단 드라이버는 항상하나를 가진다, spark context 또는 spark session 위는 session
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[2] appName=Conf>

spark session 을 가지고 작업을한다 .

In [146]:
sc.stop()

In [150]:
spark

In [155]:
#구조를 모르고 함수쓰는건 아무 짝애도 쓸모가없다